In [ ]:
%load_ext autoreload
%autoreload 2

import os
print(os.getcwd())
import sys
sys.path.append(os.path.abspath(".."))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:


import pandas as pd
from utils.data_prep import *
from utils.params import *
from utils.openai import *
from utils.score_sentiment import *
from utils.language import *
from utils.bert import BERTTopic


/root/code/Vivi-dsz/value_mapping/value_mapping/notebooks


/root/.pyenv/versions/3.12.9/envs/value_mapping/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
file_path = '/root/code/Vivi-dsz/value_mapping/value_mapping/data/raw/user_reviews_v01.csv'
reviews_df = pd.read_csv(file_path)
reviews_df

,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,app
0,f97ba6d2-4d7c-4aab-ae45-a847584972cf,was using stocard everyday. without any notice...,1,0,25.29.239,2025-07-27 12:10:19,Hello and thank you for your review.\nThe loya...,2025-07-27 12:30:43,25.29.239,Klarna
1,e325f04e-0b7b-45ae-9013-a01a016baeab,Zero (0) of my cards transfered from Stocard. ...,1,0,25.29.239,2025-07-27 12:03:24,Thank you for the review. we understand that y...,2025-07-15 23:49:02,25.29.239,Klarna
2,f26beb63-d371-4759-8af7-cf4e0bf7421f,Nobody should use it. They ruined(acquired) th...,1,0,25.29.239,2025-07-27 06:28:26,Thank you for the review. We understand that y...,2025-07-27 06:49:25,25.29.239,Klarna
3,eed09a6f-c72a-4662-bd81-8d19d73a01d6,This Klarna app is garbage. When they switched...,1,0,25.29.239,2025-07-27 05:22:54,Thanks for the review. We are sorry to hear th...,2025-07-27 05:43:29,25.29.239,Klarna
4,1bcfeb1b-ca54-4883-9c12-34df63e6e8c4,"Absolutely horrendous, cannot transfer years o...",1,0,NaN,2025-07-27 01:22:31,Thank you for the review. We try to innovate o...,2025-07-27 01:41:27,NaN,Klarna
...,...,...,...,...,...,...,...,...,...,...
4995,8e94a970-453d-48c9-9c09-495acefac2cd,All fun untill you got the end of the month an...,2,2,22.15.2,2023-07-01 16:11:03,Hey Túlio! We always strive to offer you the b...,2023-07-03 09:41:35,22.15.2,Bunq
4996,740e96d7-d98e-44f2-a869-8c4045c9a38f,I like the concept of the app. But the navigat...,4,0,22.15.2,2023-07-01 08:29:41,Hey there Angelica 👋 Thanks for the review 🚀 I...,2023-07-03 09:17:55,22.15.2,Bunq
4997,bbffd767-a724-44ec-b3d8-de9dc215424b,Great app,5,0,22.15.2,2023-07-01 03:49:05,Hey Christian 👋 Thank you for the five-star re...,2023-07-03 09:12:23,22.15.2,Bunq
4998,f2c6266c-2623-41a3-a9dd-c1e8157b1433,Amazing l like it,3,0,22.15.2,2023-07-01 00:24:33,Hey Ali 👋 Thank you for your review. We’re alw...,2023-07-03 09:11:42,22.15.2,Bunq


## Sample "About Me" Text

In [5]:
brand_text ={"Klrana" : """
Hej, we’re Klarna
We’re here to set the new standard for how people shop and pay–proudly Swedish.

Smarter shopping starts here
At Klarna, we're redefining the shopping experience to help people get more out of their money. Here's how:
Pay your way: Choose from interest-free payment plans and customizable payment options
Earn while you shop: Receive cashback directly in your Klarna balance.
Secure shopping: Keep your data safe using advanced encryption and 24/7 fraud monitoring.
Our History
Made in Stockholm
Three young Swedish entrepreneurs had a brilliant idea back in 2005—but not the best name. Kreditor became Klarna in 2010.
Unicorn status unlocked
In 2012, Klarna reached a significant milestone with a $1B+ valuation, recognizing our rapid growth and market impacts.
Klarna's family grows
In 2014, Klarna acquired SOFORT, creating Europe’s fastest-growing online payments group.
A bold new era
In 2017, Klarna transformed from blue to bold pink, and secured a full banking license, expanding our capabilities to serve customers.
The Klarna app arrives
Launched in 2018, the Klarna app provides users with tools for smarter shopping, budget management, and seamless payments across online and in-store experiences.
The smart spending leader
As of 2025, Klarna is a leader in digital payments, offering flexible payment options, cashback, and financial tools to millions of shoppers worldwide.

The numbers don't lie
100m
active consumers

720k
M erchants

2.9m
transactions per day

26
C ountries supported

Trusted by the world's most loved brands

Meet the board
The board is Klarna’s top decision-making body, overseeing strategy, operations, and corporate governance to ensure accountability to both the organization and investors.

Get to know our culture
With offices around the world, our global team blends start-up energy with a drive to create bold, impactful change and redefine smarter spending.
"""}

In [ ]:
rows = []
for brand, raw_text in brand_text.items():
    text, tokens = clean_and_tokenize(raw_text)
    rows.append({
        "brand": brand,
        "tokens": tokens
    })

df = pd.DataFrame(rows)
print(df)

## Questions ##

1. What we do with typos? Like C ountries?

## Clean Data

In [65]:
reviews_df["content_tockens"] = reviews_df["content"].apply(lambda x: clean_and_tokenize(x)[1])
reviews_df["content_cleaned"] = reviews_df["content"].apply(lambda x: clean_and_tokenize(x)[0])

In [ ]:
# path = '/root/code/Vivi-dsz/value_mapping/value_mapping/data/preprocessed/user_reviews.csv'
# reviews_df.to_csv(path)

## Topic Encoding

## 1. OpenAI API 

Check https://github.com/openai/openai-python
https://github.com/openai/openai-python?utm_source=chatgpt.com

In [ ]:
# Apply it to the DataFrame (slow if many rows)
df_sample = reviews_df.sample(150, random_state=42)

df_sample["openai_topic"] = df_sample["content"].apply(
    lambda x: get_topic_from_openai_constrained(x, values_keywords)
)


['accessibility', 'accountability', 'aesthetics', 'affordable', 'ai', 'ambition', 'automation', 'autonomy', 'bravery', 'choice', 'clarity', 'clean', 'collaboration', 'community', 'control', 'convenience', 'cooperation', 'credibility', 'customer orientation', 'customer service', 'customization', 'cutting-edge', 'data-driven', 'development', 'digital', 'disruption', 'diversity', 'ease', 'education', 'efficiency', 'empowerment', 'engagement', 'ethics', 'expansion', 'fairness', 'financial independence', 'flexibility', 'freedom', 'functional', 'future', 'global access', 'goal', 'growth', 'helpful', 'honesty', 'impact', 'inclusion', 'independence', 'innovation', 'integrity', 'internationalwealth', 'intuitive', 'inventive', 'local relevance', 'modernity', 'parity', 'personalization', 'planning', 'proactivity', 'product quality', 'protection', 'reliability', 'responsibility', 'safety', 'satisfaction', 'scalability', 'secure', 'self-determination', 'simple', 'simplicity', 'sincerity', 'smart', 

In [77]:
# Reverse the values_keywords dictionary
keyword_to_category = reverse_keyword_mapping(values_keywords)

# Now map it to your DataFrame
df_sample["topic_category"] = df_sample["openai_topic"].map(keyword_to_category)

In [78]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150 entries, 1501 to 2575
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              150 non-null    object
 1   content               150 non-null    object
 2   score                 150 non-null    int64 
 3   thumbsUpCount         150 non-null    int64 
 4   reviewCreatedVersion  121 non-null    object
 5   at                    150 non-null    object
 6   replyContent          89 non-null     object
 7   repliedAt             89 non-null     object
 8   appVersion            121 non-null    object
 9   app                   150 non-null    object
 10  content_tockens       150 non-null    object
 11  content_cleaned       150 non-null    object
 12  openai_topic          150 non-null    object
 13  topic_category        139 non-null    object
dtypes: int64(2), object(12)
memory usage: 17.6+ KB


In [79]:
df_sample.to_csv('/root/code/Vivi-dsz/value_mapping/value_mapping/data/preprocessed/openai_topics.csv')
# , encoding='utf-8',
#     sep=',', index=False)

## 2. Bert

Bert Topic includes models like OpenAI, KeyBERTInspired, MaximalMarginalRelevance
https://maartengr.github.io/BERTopic/index.html#quick-start
https://maartengr.github.io/BERTopic/getting_started/representation/representation.html


In [ ]:
#  documents are a list of cleaned review texts
docs = reviews_df["content_cleaned"].dropna().tolist()

# Initialize with your OpenAI client
berttopic = BERTTopic(docs=docs, client=client)

# Run desired topic models
berttopic.openai()
berttopic.keybert()
berttopic.mmr()

# Get combined result DataFrame
berttopic_df = berttopic.get_results()


## Evaluation

# Pipeline

In [80]:
# Detect the language of the review if necessary
reviews_df["language"] = reviews_df["content_cleaned"].apply(detect_language)

In [82]:
# Apply to score sentiment
reviews_df["label_sentiment"] = reviews_df["score"].apply(score_to_sentiment)


## Evaluation